## Synthetizer of information 

### This is a tool that can create learning material and a guide to help anyone to start to learn any topic

In [1]:
import os
import json
import requests
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [4]:
load_dotenv(override=True)
api_key_llm = os.getenv('OPENAI_API_KEY')
api_key_search = os.getenv("GOOGLE_SEARCH_API_KEY")
search_engine_id = os.getenv("GOOGLE_SEARCH_ENGINE_ID")
openai = OpenAI()
MODEL = 'gpt-5-nano'

In [5]:
def google_search(query, api_key, search_engine_id, num_results=10):
    """
    Perform a Google Programmable Search query
    
    query: Search query string
    api_key: Google API key
    search_engine_id: Programmable Search Engine ID (cx)
    num_results: Number of results to fetch (max 10 per API call)
    """

    url = "https://www.googleapis.com/customsearch/v1"

    params = {
        "q": query,
        "key": api_key,
        "cx": search_engine_id,
        "num": num_results
    }

    response = requests.get(url, params=params)
    data = response.json()
    return data

In [6]:
query = "how to build simple AI agent with python"
search_results = {}
search_results = google_search(
    query=query,
    api_key=api_key_search,
    search_engine_id=search_engine_id
    )

In [30]:
search_results['items'][1]

{'kind': 'customsearch#result',
 'title': 'My guide on what tools to use to build AI agents (if you are a newb ...',
 'htmlTitle': 'My guide on what tools to use to <b>build AI agents</b> (if you are a newb ...',
 'link': 'https://www.reddit.com/r/AI_Agents/comments/1il8b1i/my_guide_on_what_tools_to_use_to_build_ai_agents/',
 'displayLink': 'www.reddit.com',
 'snippet': 'Feb 9, 2025 ... ... Python package that enables you to build quick simple web UIs for python projects. And my last bit of advice for all newbs to Agentic Ai.',
 'htmlSnippet': 'Feb 9, 2025 <b>...</b> ... <b>Python</b> package that enables you to <b>build</b> quick <b>simple</b> web UIs for <b>python</b> projects. And my last bit of advice for all newbs to Agentic <b>Ai</b>.',
 'formattedUrl': 'https://www.reddit.com/.../AI_Agents/.../my_guide_on_what_tools_to_use_...',
 'htmlFormattedUrl': 'https://www.reddit.com/.../<b>AI</b>_<b>Agent</b>s/.../my_guide_on_what_tools_to_use_...',
 'pagemap': {'metatags': [{'og:image': 

In [9]:
search_results['items'][1]['link']

'https://www.reddit.com/r/AI_Agents/comments/1il8b1i/my_guide_on_what_tools_to_use_to_build_ai_agents/'

In [12]:
for url in search_results['items']:
    print(url['link'])

https://www.youtube.com/watch?v=bTMPwUgLZf0
https://www.reddit.com/r/AI_Agents/comments/1il8b1i/my_guide_on_what_tools_to_use_to_build_ai_agents/
https://www.youtube.com/watch?v=bZzyPscbtI8
https://medium.com/@gurpartap.sandhu3/build-you-own-ai-agent-from-scratch-in-30-mins-using-simple-python-1458f8099da0
https://www.youtube.com/watch?v=E4l91XKQSgw
https://medium.com/@jagadeesan.ganesh/mastering-llm-ai-agents-building-and-using-ai-agents-in-python-with-real-world-use-cases-c578eb640e35
https://www.youtube.com/watch?v=gaGaPpnexxA
https://learnwithhasan.com/blog/create-ai-agents-with-python/
https://medium.com/@dvasquez.422/building-a-simple-ai-agent-1e2f2b369b25
https://www.reddit.com/r/PromptEngineering/comments/1im69jz/learn_to_create_your_first_ai_agent_easily/


In [13]:
# I can call an LLM and ask it to pick the most relevant search results in terms of what I want the LLM to do

In [ ]:
link_system_prompt = f"""
You are provided with a list of 10 search results in a form of title and url links. The list will be provided in this format:
{
    "results": [{"title_1": "title", "link_1": "url"}, {"title_2": "title", "link_2": "url"}]
}
You are able to decide which of the links would be most relevant for a research that will follow. This research is aiming
to create a brief study material for beginners in this topic:
{query}
The goal is to create study material which will help beginners to start learning and help them and motivate them to continue
studying in the first crucial weeks. 
You should respond in JSON as in this example:

{
    "links": [
        {"url": "https://full.url/goes/here/"},
        {"url": "https://another.full.url/"}
    ]
}
"""

In [ ]:
# I want to construct the list of links for the user prompt
# {"results": [
# {"title": title, "link": url},
# ]}

def get_search_evaluation_links(search):
    results_for_llm = {"results": []}
    info_block = {}

    for i, link in enumerate(search['items']):
        info_block["title_" + str(i+1)] = link['title']
        info_block["link_" + str(i+1)] = link['link']
        results_for_llm["results"].append(info_block)
        info_block = {}
    return results_for_llm

In [39]:
get_search_evaluation_links(search_results)

{'results': [{'title_1': 'Build an AI Agent From Scratch in Python - Tutorial for Beginners ...',
   'link_1': 'https://www.youtube.com/watch?v=bTMPwUgLZf0'},
  {'title_2': 'My guide on what tools to use to build AI agents (if you are a newb ...',
   'link_2': 'https://www.reddit.com/r/AI_Agents/comments/1il8b1i/my_guide_on_what_tools_to_use_to_build_ai_agents/'},
  {'title_3': 'Building AI Agents in Pure Python - Beginner Course - YouTube',
   'link_3': 'https://www.youtube.com/watch?v=bZzyPscbtI8'},
  {'title_4': 'Build you own AI agent from scratch in 30 mins using simple Python ...',
   'link_4': 'https://medium.com/@gurpartap.sandhu3/build-you-own-ai-agent-from-scratch-in-30-mins-using-simple-python-1458f8099da0'},
  {'title_5': 'How to Build a Local AI Agent With Python (Ollama, LangChain ...',
   'link_5': 'https://www.youtube.com/watch?v=E4l91XKQSgw'},
  {'title_6': 'Mastering LLM AI Agents: Building and Using AI Agents in Python ...',
   'link_6': 'https://medium.com/@jagadees

In [42]:
def get_user_prompt(query, api_key, search_engine_id, num_results=10):
    search_results = google_search(query, api_key, search_engine_id, num_results=10)
    links_for_llm = get_search_evaluation_links(search_results)
    user_prompt = f"""
    Here is the list of titles and links that resulted from the web search:
    {links_for_llm}
    Please decide which of these are the most relevant web links that could be further examined to create
    a brief study material for beginners in this topic:
    {query}.
    The goal is to create study material which will help beginners to start learning and help them and motivate them to continue
    studying in the first crucial weeks.
    """
    return user_prompt


    

In [41]:
prompt = get_user_prompt(query, api_key_search, search_engine_id, num_results=10)
print(prompt)


    Here is the list of titles and links that resulted from the web search:
    {'results': [{'title_1': 'Build an AI Agent From Scratch in Python - Tutorial for Beginners ...', 'link_1': 'https://www.youtube.com/watch?v=bTMPwUgLZf0'}, {'title_2': 'My guide on what tools to use to build AI agents (if you are a newb ...', 'link_2': 'https://www.reddit.com/r/AI_Agents/comments/1il8b1i/my_guide_on_what_tools_to_use_to_build_ai_agents/'}, {'title_3': 'Building AI Agents in Pure Python - Beginner Course - YouTube', 'link_3': 'https://www.youtube.com/watch?v=bZzyPscbtI8'}, {'title_4': 'Mastering LLM AI Agents: Building and Using AI Agents in Python ...', 'link_4': 'https://medium.com/@jagadeesan.ganesh/mastering-llm-ai-agents-building-and-using-ai-agents-in-python-with-real-world-use-cases-c578eb640e35'}, {'title_5': 'How to Build a Local AI Agent With Python (Ollama, LangChain ...', 'link_5': 'https://www.youtube.com/watch?v=E4l91XKQSgw'}, {'title_6': 'Build you own AI agent from scratch i

In [ ]:
messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_user_prompt(query, api_key_search, search_engine_id, num_results=10)}
        ]